In [10]:
"""
Demonstration of rotational slippage using the Screw-Panda framework.
This example shows how to rotate a box around a screw axis that passes
through the bottom back edge, and computes the corresponding end-effector
trajectory for a Panda robot.
"""
# We will do the imports required for this notebook here
%load_ext autoreload
%autoreload 2
import numpy as np
from spatialmath import SE3, SO3
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import roboticstoolbox as rtb
import pytransform3d.transformations as pt3d

# Import from Screw-Panda framework
from utils.objects import ManipulableBox, SpatialObject
from utils.screw_utils import (
    generate_trajectory, 
    pose_to_screw_parameters, 
    twist_from_poses
)

In [16]:
def create_box_and_screw_axis():
    """Create a box and define a screw axis through its bottom back edge."""
    # Create box with dimensions (x, y, z) in meters
    box = ManipulableBox(dimensions=(0.1, 0.07, 0.03), name="target_box")
    
    # Initial pose - slightly above the ground
    initial_pose = SE3(0.5, 0, 0.015)
    box.update_pose(initial_pose)
    
    
    # Get the bottom back edge (edge 1 in the ManipulableBox class)
    
    edge_start, edge_end = box.get_edge_in_world(1)

    # Middle point of the edge as the point on the screw axis
    point_on_axis = box.get_edge_contact_pose(edge_idx=1, contact_param=0.5).t

    # Direction of the edge - this will be our screw axis direction
    axis_direction = edge_end - edge_start
    axis_direction = axis_direction / np.linalg.norm(axis_direction)
    
    print(f"Box dimensions: {box.length} x {box.width} x {box.height} meters")
    print(f"Initial pose: \n{box.pose}")
    print(f"Bottom back edge: from \n{edge_start.shape} to \n{edge_end}")
    print(f"Point on screw axis: \n{point_on_axis}")
    print(f"Screw axis direction: \n{axis_direction}")
    
    return box, point_on_axis, axis_direction


In [17]:
def create_screw_rotation(box, point_on_axis, axis_direction, angle_deg=45):
    """
    Create a screw motion that rotates the box around the defined axis.
    
    Args:
        box: The ManipulableBox object
        point_on_axis: A point on the screw axis
        axis_direction: Direction vector of the screw axis
        angle_deg: Rotation angle in degrees
        
    Returns:
        Final pose after rotation
    """
    # Convert angle to radians
    angle_rad = np.radians(angle_deg)
    
    # Create a rotation matrix around the axis direction
    # Use SO3.AngleAxis to create rotation around arbitrary axis
    R = SO3.AngleAxis(angle_rad, axis_direction).R
    
    # Current pose of the box
    start_pose = box.pose
    
    # We need to transform the rotation to be around the point on the axis
    # rather than around the origin
    p = point_on_axis
    
    # Translation part: p - R*p gives the translation needed
    # to keep the point p fixed during rotation
    t = p - R @ p
    
    # Create the final transformation
    end_pose = SE3(R, t) * start_pose
    
    return end_pose



In [18]:
# Create a box and define the screw axis
box, point_on_axis, axis_direction = create_box_and_screw_axis()

# Create the final pose after rotation
angle_deg = 70  # Rotation angle in degrees
final_pose = create_screw_rotation(box, point_on_axis, axis_direction, angle_deg)


Box dimensions: 0.1 x 0.07 x 0.03 meters
Initial pose: 
   1         0         0         0.5       
   0         1         0         0         
   0         0         1         0.015     
   0         0         0         1         

Bottom back edge: from 
(3,) to 
[ 0.45  -0.035  0.   ]
Point on screw axis: 
[0.45 0.   0.  ]
Screw axis direction: 
[ 0. -1.  0.]


ValueError: Invalid arguments. See documentation for correct format.